## Introducere 

Această lecție va acoperi: 
- Ce este apelarea funcțiilor și cazurile lor de utilizare 
- Cum să creezi un apel de funcție folosind OpenAI 
- Cum să integrezi un apel de funcție într-o aplicație 

## Obiective de învățare 

După finalizarea acestei lecții vei ști cum să și vei înțelege: 

- Scopul utilizării apelării funcțiilor 
- Configurarea apelului de funcție folosind serviciul OpenAI 
- Proiectarea apelurilor de funcții eficiente pentru cazul tău de utilizare în aplicații 


## Înțelegerea Apelurilor de Funcții

Pentru această lecție, dorim să construim o funcționalitate pentru startup-ul nostru educațional care să permită utilizatorilor să folosească un chatbot pentru a găsi cursuri tehnice. Vom recomanda cursuri care se potrivesc nivelului lor de competență, rolului actual și tehnologiei de interes.

Pentru a realiza acest lucru, vom folosi o combinație de:
 - `OpenAI` pentru a crea o experiență de chat pentru utilizator
 - `Microsoft Learn Catalog API` pentru a ajuta utilizatorii să găsească cursuri pe baza cererii utilizatorului
 - `Function Calling` pentru a prelua interogarea utilizatorului și a o trimite către o funcție care să facă cererea API.

Pentru a începe, să vedem de ce am dori să folosim apelul de funcții în primul rând:

print("Mesaje în următoarea cerere:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obține un răspuns nou de la GPT unde poate vedea răspunsul funcției


print(second_response.choices[0].message)


### De ce apelarea funcțiilor

Dacă ați finalizat orice altă lecție din acest curs, probabil că înțelegeți puterea utilizării modelelor mari de limbaj (LLM-uri). Sperăm că puteți vedea și unele dintre limitările lor.

Apelarea funcțiilor este o caracteristică a serviciului OpenAI concepută pentru a aborda următoarele provocări:

Formatare inconsistentă a răspunsurilor:
- Înainte de apelarea funcțiilor, răspunsurile unui model mare de limbaj erau neorganizate și inconsistente. Dezvoltatorii trebuiau să scrie cod complex de validare pentru a gestiona fiecare variație a rezultatului.

Integrare limitată cu date externe:
- Înainte de această caracteristică, era dificil să se încorporeze date din alte părți ale unei aplicații într-un context de chat.

Prin standardizarea formatelor de răspuns și facilitarea integrării fără probleme cu date externe, apelarea funcțiilor simplifică dezvoltarea și reduce necesitatea logicii suplimentare de validare.

Utilizatorii nu puteau obține răspunsuri precum „Care este vremea actuală în Stockholm?”. Acest lucru se datorează faptului că modelele erau limitate la momentul în care datele au fost antrenate.

Să analizăm exemplul de mai jos care ilustrează această problemă:

Să presupunem că dorim să creăm o bază de date cu date despre studenți pentru a le putea sugera cursul potrivit. Mai jos avem două descrieri ale studenților care sunt foarte asemănătoare în datele pe care le conțin.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Dorim să trimitem acest lucru către un LLM pentru a analiza datele. Acest lucru poate fi folosit ulterior în aplicația noastră pentru a trimite aceste informații către un API sau pentru a le stoca într-o bază de date.

Să creăm două prompturi identice în care să instruim LLM-ul despre ce informații ne interesează:


Vrem să trimitem acest lucru unui LLM pentru a analiza părțile importante pentru produsul nostru. Astfel, putem crea două prompturi identice pentru a instrui LLM-ul:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


După ce creăm aceste două prompturi, le vom trimite către LLM folosind `openai.ChatCompletion`. Stocăm promptul în variabila `messages` și atribuim rolul `user`. Acest lucru este pentru a imita un mesaj scris de un utilizator către un chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Acum putem trimite ambele cereri către LLM și să examinăm răspunsul pe care îl primim.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Chiar dacă prompturile sunt aceleași și descrierile sunt similare, putem obține formate diferite pentru proprietatea `Grades`.

Dacă rulezi celula de mai sus de mai multe ori, formatul poate fi `3.7` sau `3.7 GPA`.

Acest lucru se datorează faptului că LLM preia date nestructurate sub forma promptului scris și returnează, de asemenea, date nestructurate. Avem nevoie de un format structurat pentru a ști la ce să ne așteptăm atunci când stocăm sau folosim aceste date.

Folosind apelarea funcțională, ne putem asigura că primim înapoi date structurate. Când folosim apelarea funcțională, LLM nu apelează sau nu rulează efectiv nicio funcție. În schimb, creăm o structură pe care LLM trebuie să o urmeze pentru răspunsurile sale. Apoi folosim acele răspunsuri structurate pentru a ști ce funcție să rulăm în aplicațiile noastre.


![Diagrama fluxului apelării funcției](../../../../translated_images/Function-Flow.083875364af4f4bb.ro.png)


Putem apoi lua ceea ce este returnat de funcție și trimite acest lucru înapoi către LLM. LLM va răspunde apoi folosind limbaj natural pentru a răspunde la întrebarea utilizatorului.


### Cazuri de utilizare pentru apelarea funcțiilor

**Apelarea uneltelor externe**  
Chatboții sunt excelenți în a oferi răspunsuri la întrebările utilizatorilor. Prin utilizarea apelării funcțiilor, chatboții pot folosi mesajele utilizatorilor pentru a îndeplini anumite sarcini. De exemplu, un student poate cere chatbotului să „Trimită un email instructorului meu spunând că am nevoie de mai mult ajutor cu acest subiect”. Acesta poate face un apel de funcție către `send_email(to: string, body: string)`

**Crearea de interogări API sau baze de date**  
Utilizatorii pot găsi informații folosind limbaj natural care este convertit într-o interogare formatată sau o cerere API. Un exemplu ar putea fi un profesor care solicită „Cine sunt studenții care au finalizat ultima temă” ceea ce ar putea apela o funcție numită `get_completed(student_name: string, assignment: int, current_status: string)`

**Crearea de date structurate**  
Utilizatorii pot lua un bloc de text sau CSV și pot folosi LLM pentru a extrage informații importante din acesta. De exemplu, un student poate converti un articol Wikipedia despre acorduri de pace pentru a crea fișe AI de studiu. Acest lucru se poate face folosind o funcție numită `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Crearea primei apelări a funcției tale

Procesul de creare a unei apelări a funcției include 3 pași principali:  
1. Apelarea API-ului Chat Completions cu o listă a funcțiilor tale și un mesaj de la utilizator  
2. Citirea răspunsului modelului pentru a efectua o acțiune, adică executarea unei funcții sau a unui apel API  
3. Efectuarea unui alt apel la API-ul Chat Completions cu răspunsul funcției tale pentru a folosi acea informație în crearea unui răspuns pentru utilizator.


![Fluxul unui apel de funcție](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.ro.png)


### Elemente ale unui apel de funcție 

#### Intrarea utilizatorului 

Primul pas este să creați un mesaj de utilizator. Acesta poate fi atribuit dinamic preluând valoarea unui câmp de text sau puteți atribui o valoare aici. Dacă este prima dată când lucrați cu API-ul Chat Completions, trebuie să definim `role` și `content` ale mesajului. 

`role` poate fi fie `system` (crearea regulilor), `assistant` (modelul) sau `user` (utilizatorul final). Pentru apelarea funcțiilor, vom atribui acestuia valoarea `user` și o întrebare exemplu. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Crearea funcțiilor.

În continuare vom defini o funcție și parametrii acelei funcții. Vom folosi doar o funcție aici numită `search_courses`, dar poți crea mai multe funcții.

**Important** : Funcțiile sunt incluse în mesajul sistem către LLM și vor fi incluse în cantitatea de tokeni disponibili pe care îi ai la dispoziție.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definiții** 

Structura definiției funcției are mai multe niveluri, fiecare cu propriile proprietăți. Iată o defalcare a structurii imbricate:

**Proprietăți la nivelul superior al funcției:**

`name` - Numele funcției pe care dorim să o apelăm. 

`description` - Aceasta este descrierea modului în care funcția funcționează. Aici este important să fim specifici și clari. 

`parameters` - O listă de valori și formatul pe care doriți ca modelul să le producă în răspunsul său. 

**Proprietăți ale obiectului Parameters:**

`type` - Tipul de date al obiectului parameters (de obicei "object")

`properties` - Lista valorilor specifice pe care modelul le va folosi pentru răspunsul său. 

**Proprietăți individuale ale parametrilor:**

`name` - Definit implicit de cheia proprietății (de exemplu, "role", "product", "level")

`type` - Tipul de date al acestui parametru specific (de exemplu, "string", "number", "boolean") 

`description` - Descrierea parametrului specific. 

**Proprietăți opționale:**

`required` - Un tablou care listează care parametri sunt necesari pentru ca apelul funcției să fie completat. 


### Apelarea funcției  
După definirea unei funcții, trebuie acum să o includem în apelul către API-ul Chat Completion. Facem acest lucru adăugând `functions` la cerere. În acest caz, `functions=functions`. 

Există, de asemenea, o opțiune de a seta `function_call` la `auto`. Aceasta înseamnă că vom lăsa LLM să decidă ce funcție ar trebui apelată pe baza mesajului utilizatorului, în loc să o atribuim noi înșine.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Acum să analizăm răspunsul și să vedem cum este formatat:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Puteți observa că numele funcției este apelat și, din mesajul utilizatorului, LLM a reușit să găsească datele pentru a completa argumentele funcției.


## 3.Integrarea apelurilor funcțiilor într-o aplicație. 


După ce am testat răspunsul formatat de la LLM, acum îl putem integra într-o aplicație. 

### Gestionarea fluxului 

Pentru a integra acest lucru în aplicația noastră, să urmăm pașii următori: 

Mai întâi, să facem apelul către serviciile OpenAI și să stocăm mesajul într-o variabilă numită `response_message`. 


In [ ]:
response_message = response.choices[0].message

Acum vom defini funcția care va apela API-ul Microsoft Learn pentru a obține o listă de cursuri:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Ca o bună practică, vom vedea apoi dacă modelul dorește să apeleze o funcție. După aceea, vom crea una dintre funcțiile disponibile și o vom potrivi cu funcția care este apelată.  
Apoi vom lua argumentele funcției și le vom mapa la argumentele din LLM.

În final, vom adăuga mesajul apelului funcției și valorile care au fost returnate de mesajul `search_courses`. Acest lucru oferă LLM toate informațiile de care are nevoie pentru a răspunde utilizatorului folosind limbaj natural.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Acum vom trimite mesajul actualizat către LLM pentru a putea primi un răspuns în limbaj natural în loc de un răspuns formatat JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Provocare de cod 

Bravo! Pentru a continua învățarea despre OpenAI Function Calling, poți construi: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Mai mulți parametri ai funcției care ar putea ajuta cursanții să găsească mai multe cursuri. Poți găsi parametrii API disponibili aici: 
 - Creează un alt apel de funcție care să preia mai multe informații de la cursant, cum ar fi limba lor maternă 
 - Creează gestionarea erorilor atunci când apelul funcției și/sau apelul API nu returnează niciun curs potrivit 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim pentru acuratețe, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă trebuie considerat sursa autorizată. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite rezultate din utilizarea acestei traduceri.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
